In [2]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score, cross_validate, cross_val_predict, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, make_scorer
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
import xgboost as xgb
from sklearn.linear_model import ElasticNet
import lightgbm as lgb
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.decomposition import PCA 

Dataset con filtro de las features elegidas:

In [18]:
# Cargar el DataFrame desde el archivo
with open('../Pickles/df_final.pickle', 'rb') as archivo:
    df = pickle.load(archivo)
# Establecer la opción para mostrar todas las columnas
pd.set_option('display.max_columns', None)

nombres=df.drop(['CompTotal'], axis=1).columns

limite_inferior = 18000
limite_superior = 100000

df_filtrado = df[(df['CompTotal'] >= limite_inferior) & (df['CompTotal'] <= limite_superior)]

print(f"Tamaño del DataFrame original: {df.shape}")
print(f"Tamaño del DataFrame filtrado: {df_filtrado.shape}")

df = df_filtrado

df['CompTotal'] = np.log1p(df['CompTotal'])
y = df['CompTotal']
df = df.drop(columns=['CompTotal'], axis=1)

Tamaño del DataFrame original: (1934, 397)
Tamaño del DataFrame filtrado: (1743, 397)


In [19]:
df= df[['MainBranch','EdLevel','YearsCode','YearsCodePro','AISelect', 'AISent',  'Frequency_1',
        'Frequency_2','Frequency_3', 'Age_Grouped_Adulto','Age_Grouped_Joven','Age_Grouped_Senior', 
        'is_full_time', 'is_part_time', 'is_independent', 'num_jobs', 'is_other_employment', 
        'CodingActivities_encoded', 'LearnCode_encoded', 'LearnCodeOnline_encoded', 
        'DevType_encoded', 'Android-based', 'Linux-based', 'Windows-based',
        'Industry_Category_Industria y Energía','Industry_Category_Otros Servicios',
        'Industry_Category_Salud y Educación','Industry_Category_Servicios Financieros',
        'Industry_Category_Tecnología y Servicios Digitales', 'BuyNewTool_Ask a generative AI tool',
        'BuyNewTool_Ask developers I know/work with','BuyNewTool_Other','BuyNewTool_Other (please specify):',
        'BuyNewTool_Read ratings or reviews on third party sites like G2 Crowd',
        'BuyNewTool_Research companies that have advertised on sites I visit',
        'BuyNewTool_Research companies that have emailed me','BuyNewTool_Start a free trial',
        'BuyNewTool_Visit developer communities like Stack Overflow', 'LanguageHaveWorkedWith_', 
        'LanguageHaveWorkedWith_Ada', 'LanguageHaveWorkedWith_Apex', 'LanguageHaveWorkedWith_Assembly',
        'LanguageHaveWorkedWith_Bash/Shell (all shells)', 'LanguageHaveWorkedWith_C', 
        'LanguageHaveWorkedWith_C#','LanguageHaveWorkedWith_C++', 'LanguageHaveWorkedWith_Clojure', 
        'LanguageHaveWorkedWith_Cobol','LanguageHaveWorkedWith_Crystal','LanguageHaveWorkedWith_Dart',
        'LanguageHaveWorkedWith_Delphi','LanguageHaveWorkedWith_Elixir','LanguageHaveWorkedWith_Erlang',
        'LanguageHaveWorkedWith_F#','LanguageHaveWorkedWith_Fortran','LanguageHaveWorkedWith_GDScript',
        'LanguageHaveWorkedWith_Go','LanguageHaveWorkedWith_Groovy','LanguageHaveWorkedWith_HTML/CSS',
        'LanguageHaveWorkedWith_Haskell','LanguageHaveWorkedWith_Java','LanguageHaveWorkedWith_JavaScript',
        'LanguageHaveWorkedWith_Julia','LanguageHaveWorkedWith_Kotlin','LanguageHaveWorkedWith_Lisp',
        'LanguageHaveWorkedWith_Lua', 'LanguageHaveWorkedWith_MATLAB','LanguageHaveWorkedWith_Nim', 
        'LanguageHaveWorkedWith_OCaml','LanguageHaveWorkedWith_Objective-C','LanguageHaveWorkedWith_PHP', 
        'LanguageHaveWorkedWith_Perl', 'LanguageHaveWorkedWith_PowerShell', 'LanguageHaveWorkedWith_Prolog',
        'LanguageHaveWorkedWith_Python', 'LanguageHaveWorkedWith_R', 'LanguageHaveWorkedWith_Ruby', 
        'LanguageHaveWorkedWith_Rust', 'LanguageHaveWorkedWith_SQL', 'LanguageHaveWorkedWith_Scala', 
        'LanguageHaveWorkedWith_Solidity', 'LanguageHaveWorkedWith_Swift', 'LanguageHaveWorkedWith_TypeScript', 
        'LanguageHaveWorkedWith_VBA', 'LanguageHaveWorkedWith_Visual Basic (.Net)', 
        'LanguageHaveWorkedWith_Zig', 'LanguageWantToWorkWith_', 'LanguageWantToWorkWith_Ada', 
        'LanguageWantToWorkWith_Apex', 'LanguageWantToWorkWith_Assembly', 'LanguageWantToWorkWith_Bash/Shell (all shells)',
        'LanguageWantToWorkWith_C', 'LanguageWantToWorkWith_C#', 'LanguageWantToWorkWith_C++', 
        'LanguageWantToWorkWith_Clojure', 'LanguageWantToWorkWith_Crystal', 'LanguageWantToWorkWith_Dart', 
        'LanguageWantToWorkWith_Delphi', 'LanguageWantToWorkWith_Elixir', 'LanguageWantToWorkWith_Erlang', 
        'LanguageWantToWorkWith_F#', 'LanguageWantToWorkWith_Fortran', 'LanguageWantToWorkWith_GDScript', 
        'LanguageWantToWorkWith_Go', 'LanguageWantToWorkWith_Groovy', 'LanguageWantToWorkWith_HTML/CSS',
        'LanguageWantToWorkWith_Haskell', 'LanguageWantToWorkWith_Java', 'LanguageWantToWorkWith_JavaScript', 
        'LanguageWantToWorkWith_Julia', 'LanguageWantToWorkWith_Kotlin', 'LanguageWantToWorkWith_Lisp', 
        'LanguageWantToWorkWith_Lua', 'LanguageWantToWorkWith_MATLAB', 'LanguageWantToWorkWith_Nim', 
        'LanguageWantToWorkWith_OCaml', 'LanguageWantToWorkWith_Objective-C', 'LanguageWantToWorkWith_PHP', 
        'LanguageWantToWorkWith_Perl', 'LanguageWantToWorkWith_PowerShell', 
        'LanguageWantToWorkWith_Prolog', 'LanguageWantToWorkWith_Python','LanguageWantToWorkWith_R',
        'LanguageWantToWorkWith_Ruby', 'LanguageWantToWorkWith_Rust', 'LanguageWantToWorkWith_SQL', 
        'LanguageWantToWorkWith_Scala','LanguageWantToWorkWith_Solidity', 'LanguageWantToWorkWith_Swift', 
        'LanguageWantToWorkWith_TypeScript', 'LanguageWantToWorkWith_VBA', 
        'LanguageWantToWorkWith_Visual Basic (.Net)','LanguageWantToWorkWith_Zig', 
        'DatabaseHaveWorkedWith_', 'DatabaseHaveWorkedWith_BigQuery', 'DatabaseHaveWorkedWith_Cassandra', 
        'DatabaseHaveWorkedWith_Clickhouse', 'DatabaseHaveWorkedWith_Cloud Firestore', 
        'DatabaseHaveWorkedWith_Cockroachdb', 'DatabaseHaveWorkedWith_Cosmos DB', 
        'DatabaseHaveWorkedWith_Couch DB', 'DatabaseHaveWorkedWith_Couchbase', 
        'DatabaseHaveWorkedWith_Datomic', 'DatabaseHaveWorkedWith_DuckDB', 'DatabaseHaveWorkedWith_Dynamodb', 
        'DatabaseHaveWorkedWith_Elasticsearch', 'DatabaseHaveWorkedWith_Firebase Realtime Database', 
        'DatabaseHaveWorkedWith_Firebird', 'DatabaseHaveWorkedWith_H2', 'DatabaseHaveWorkedWith_IBM DB2', 
        'DatabaseHaveWorkedWith_InfluxDB', 'DatabaseHaveWorkedWith_MariaDB' , 'DatabaseHaveWorkedWith_Microsoft Access', 
        'DatabaseHaveWorkedWith_Microsoft SQL Server', 'DatabaseHaveWorkedWith_MongoDB', 'DatabaseHaveWorkedWith_MySQL', 
        'DatabaseHaveWorkedWith_Neo4J', 'DatabaseHaveWorkedWith_Oracle', 'DatabaseHaveWorkedWith_PostgreSQL', 
        'DatabaseHaveWorkedWith_RavenDB', 'DatabaseHaveWorkedWith_Redis', 'DatabaseHaveWorkedWith_SQLite', 
        'DatabaseHaveWorkedWith_Snowflake', 'DatabaseHaveWorkedWith_Solr', 'DatabaseHaveWorkedWith_Supabase',
        'ToolsTechHaveWorkedWith_', 'ToolsTechHaveWorkedWith_APT', 'ToolsTechHaveWorkedWith_Ansible', 
        'ToolsTechHaveWorkedWith_Ant', 'ToolsTechHaveWorkedWith_Bun', 'ToolsTechHaveWorkedWith_Chef', 
        'ToolsTechHaveWorkedWith_Chocolatey', 'ToolsTechHaveWorkedWith_Composer', 'ToolsTechHaveWorkedWith_Dagger', 
        'ToolsTechHaveWorkedWith_Docker', 'ToolsTechHaveWorkedWith_Godot','ToolsTechHaveWorkedWith_Google Test', 
        'ToolsTechHaveWorkedWith_Gradle', 'ToolsTechHaveWorkedWith_Homebrew', 'ToolsTechHaveWorkedWith_Kubernetes', 
        'ToolsTechHaveWorkedWith_MSBuild', 'ToolsTechHaveWorkedWith_Make', 'ToolsTechHaveWorkedWith_Maven (build tool)', 
        'ToolsTechHaveWorkedWith_Ninja', 'ToolsTechHaveWorkedWith_Nix', 'ToolsTechHaveWorkedWith_NuGet', 'ToolsTechHaveWorkedWith_Pacman', 
        'ToolsTechHaveWorkedWith_Pip', 'ToolsTechHaveWorkedWith_Podman', 'ToolsTechHaveWorkedWith_Pulumi', 'ToolsTechHaveWorkedWith_Puppet', 
        'ToolsTechHaveWorkedWith_Terraform', 'ToolsTechHaveWorkedWith_Unity 3D', 'ToolsTechHaveWorkedWith_Unreal Engine', 
        'ToolsTechHaveWorkedWith_Visual Studio Solution', 'ToolsTechHaveWorkedWith_Vite', 'ToolsTechHaveWorkedWith_Webpack',
          'ToolsTechHaveWorkedWith_Yarn', 'ToolsTechHaveWorkedWith_npm', 'ToolsTechHaveWorkedWith_pnpm']]
# Dividir los datos en train y test
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Reducir las columnas a la mitad con PCA
n_columns = X_train.shape[1]  # Número de columnas originales
pca = PCA(n_components=n_columns // 2)  # Reducir a la mitad
X_train_reduced = pca.fit_transform(X_train)
X_test_reduced = pca.transform(X_test)

Primer modelo: baseline

In [12]:
# Asegúrate de convertir los arrays de numpy a pandas DataFrame y Series si es necesario
X_train = pd.DataFrame(X_train)  # Si X_train es un ndarray, conviértelo en DataFrame
y_train = pd.Series(y_train)  # Si y_train es un ndarray, conviértelo en Series

# Crear un modelo de Random Forest
rf_model = RandomForestRegressor(random_state=42)

cv = 5  # Número de folds

# Inicializamos listas para guardar las métricas de cada fold
r2_scores = []
mse_scores = []
mae_scores = []
mape_scores = []

# Realizar la validación cruzada
kf = KFold(n_splits=cv, shuffle=True, random_state=42)
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]  # Usar .iloc con DataFrame
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]  # Usar .iloc con Series
    
    # Entrenar el modelo en el fold
    rf_model.fit(X_train_fold, y_train_fold)
    
    # Predecir en el conjunto de validación
    y_val_pred_log = rf_model.predict(X_val_fold)
    
    # Revertir la transformación logarítmica
    y_val_pred = np.expm1(y_val_pred_log)
    y_val_actual = np.expm1(y_val_fold)
    
    # Calcular las métricas para este fold
    r2 = r2_score(y_val_actual, y_val_pred)
    mse = mean_squared_error(y_val_actual, y_val_pred)
    mae = mean_absolute_error(y_val_actual, y_val_pred)
    mape = np.mean(np.abs((y_val_actual - y_val_pred) / y_val_actual)) * 100  
    
    # Guardar las métricas en las listas
    r2_scores.append(r2)
    mse_scores.append(mse)
    mae_scores.append(mae)
    mape_scores.append(mape)  # Guardar MAPE

# Promedios de métricas en CV
mean_r2_cv = np.mean(r2_scores)
mean_mse_cv = np.mean(mse_scores)
mean_rmse_cv = np.sqrt(mean_mse_cv)
mean_mae_cv = np.mean(mae_scores)
mean_mape_cv = np.mean(mape_scores)  # Promedio de MAPE

# Métricas de Cross-Validation
print("=== Métricas de Cross-Validation ===")
print(f"R² (promedio CV): {mean_r2_cv:.4f}")
print(f"MSE (promedio CV): {mean_mse_cv:.4f}")
print(f"RMSE (promedio CV): {mean_rmse_cv:.4f}")
print(f"MAE (promedio CV): {mean_mae_cv:.4f}")
print(f"MAPE (promedio CV): {mean_mape_cv:.4f}%")  # Imprimir MAPE promedio

=== Métricas de Cross-Validation ===
R² (promedio CV): 0.5941
MSE (promedio CV): 211685540.0328
RMSE (promedio CV): 14549.4172
MAE (promedio CV): 9992.2734
MAPE (promedio CV): 20.0289%


El Linear Regressor da bastante bien, pruebo regularizandolo.

In [20]:
# Asegúrate de que X_train y y_train sean DataFrame y Series de pandas
X_train = pd.DataFrame(X_train)
y_train = pd.Series(y_train)

# Definir el rango de valores de alpha para la búsqueda en cuadrícula
param_grid = {'alpha': np.logspace(-6, 6, 13)}  # Valores de alpha entre 0.0001 y 10000

# Inicialización del modelo Ridge
ridge_model = Ridge(random_state=42)

# Configuración de GridSearchCV para encontrar el mejor alpha
grid_search = GridSearchCV(estimator=ridge_model, param_grid=param_grid, 
                           scoring='neg_mean_squared_error', cv=5, n_jobs=-1)

# Ajustar el modelo a los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Imprimir el mejor valor de alpha
print(f"Mejor valor de alpha: {grid_search.best_params_['alpha']}")

# Mejor modelo después de GridSearchCV
best_ridge_model = grid_search.best_estimator_

# Realizar la validación cruzada
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Listas para almacenar las métricas de cada fold
r2_scores = []
mse_scores = []
rmse_scores = []
mae_scores = []
mape_scores = []

for train_index, val_index in cv.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

    # Entrenar el mejor modelo Ridge en el fold actual
    best_ridge_model.fit(X_train_fold, y_train_fold)

    # Hacer predicciones en el fold de validación (en la escala logarítmica)
    y_val_pred_log = best_ridge_model.predict(X_val_fold)

    # Aplicar la transformación inversa a las predicciones y a los valores reales
    y_val_pred = np.expm1(y_val_pred_log)
    y_val_original = np.expm1(y_val_fold)

    # Calcular métricas en la escala original
    r2_scores.append(r2_score(y_val_original, y_val_pred))
    mse_scores.append(mean_squared_error(y_val_original, y_val_pred))
    rmse_scores.append(np.sqrt(mse_scores[-1]))
    mae_scores.append(mean_absolute_error(y_val_original, y_val_pred))
    mape_scores.append(np.mean(np.abs((y_val_original - y_val_pred) / y_val_original)) * 100)

# Calcular los promedios de las métricas
mean_r2_cv = np.mean(r2_scores)
mean_mse_cv = np.mean(mse_scores)
mean_rmse_cv = np.mean(rmse_scores)
mean_mae_cv = np.mean(mae_scores)
mean_mape_cv = np.mean(mape_scores)

# Imprimir las métricas
print("=== Métricas de Cross-Validation (Ridge) ===")
print(f"R² (promedio CV): {mean_r2_cv:.4f}")
print(f"MSE (promedio CV): {mean_mse_cv:.4f}")
print(f"RMSE (promedio CV): {mean_rmse_cv:.4f}")
print(f"MAE (promedio CV): {mean_mae_cv:.4f}")
print(f"MAPE (promedio CV): {mean_mape_cv:.4f}%")

Mejor valor de alpha: 1.0
=== Métricas de Cross-Validation (Ridge) ===
R² (promedio CV): 0.4101
MSE (promedio CV): 235957091.5062
RMSE (promedio CV): 15306.5036
MAE (promedio CV): 11334.6724
MAPE (promedio CV): 24.5779%


No funciona el linear regressor, las relaciones entre los datos son mas complejas. Probamos con Modelos complejos

Pruebo Gridsearch de Random Forest para mejorar los hiperparametros

In [21]:
from sklearn.metrics import make_scorer
X_train = pd.DataFrame(X_train)
y_train = pd.Series(y_train)

# Definir MAPE como función
def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Crear el scorer
mape_scorer = make_scorer(mape, greater_is_better=False)

param_grid = {
    'n_estimators': [300],
    'max_depth': [None],
    'min_samples_split': [2],
    'min_samples_leaf': [1, 2]
}

# Crear un modelo de Random Forest
rf_model = RandomForestRegressor(random_state=42)

grid_search = GridSearchCV(
    estimator=rf_model,
    param_grid=param_grid,
    cv=3,
    scoring={
        'r2': 'r2',
        'neg_mse': 'neg_mean_squared_error',
        'neg_mae': 'neg_mean_absolute_error',
        'neg_mape': mape_scorer  # Usar la función MAPE definida arriba
    },
    refit='neg_mape',  # Reajustar el mejor modelo basado en MAPE
    n_jobs=-1,
    verbose=1
)
# Inicializamos listas para guardar las métricas de cada fold
r2_scores = []
mse_scores = []
mae_scores = []
mape_scores = []

# Realizar la validación cruzada
kf = KFold(n_splits=3, shuffle=True, random_state=42)
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]  # Usar .iloc con DataFrame
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]  # Usar .iloc con Series
    
    # Entrenar el modelo en el fold
    grid_search.fit(X_train_fold, y_train_fold)
    
    # Predecir en el conjunto de validación
    y_val_pred_log = grid_search.predict(X_val_fold)
    
    # Revertir la transformación logarítmica
    y_val_pred = np.expm1(y_val_pred_log)
    y_val_actual = np.expm1(y_val_fold)
    
    # Calcular las métricas para este fold
    r2 = r2_score(y_val_actual, y_val_pred)
    mse = mean_squared_error(y_val_actual, y_val_pred)
    mae = mean_absolute_error(y_val_actual, y_val_pred)
    mape = np.mean(np.abs((y_val_actual - y_val_pred) / y_val_actual)) * 100

    # Guardar las métricas en las listas
    r2_scores.append(r2)
    mse_scores.append(mse)
    mae_scores.append(mae)
    mape_scores.append(mape) 
# Obtener el mejor modelo
best_rf_model = grid_search.best_estimator_

# Obtener los mejores parámetros
best_params = grid_search.best_params_
print("Mejores parámetros:", best_params)

# Promedios de métricas en CV
mean_r2_cv = np.mean(r2_scores)
mean_mse_cv = np.mean(mse_scores)
mean_rmse_cv = np.sqrt(mean_mse_cv)
mean_mae_cv = np.mean(mae_scores)
mean_mape_cv = np.mean(mape_scores) 

# Métricas de Cross-Validation
print("=== Métricas de Cross-Validation ===")
print(f"R² (promedio CV): {mean_r2_cv:.4f}")
print(f"MSE (promedio CV): {mean_mse_cv:.4f}")
print(f"RMSE (promedio CV): {mean_rmse_cv:.4f}")
print(f"MAE (promedio CV): {mean_mae_cv:.4f}")
print(f"MAPE (promedio CV): {mean_mape_cv:.4f}%")

Fitting 3 folds for each of 2 candidates, totalling 6 fits
Fitting 3 folds for each of 2 candidates, totalling 6 fits
Fitting 3 folds for each of 2 candidates, totalling 6 fits
Mejores parámetros: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}
=== Métricas de Cross-Validation ===
R² (promedio CV): 0.5530
MSE (promedio CV): 180254301.2373
RMSE (promedio CV): 13425.8818
MAE (promedio CV): 9468.7451
MAPE (promedio CV): 20.0433%


Pruebo con XGBoosting

In [12]:
# Asegúrate de que X_train y y_train sean DataFrame y Series de pandas
X_train = pd.DataFrame(X_train)  # Si X_train es un ndarray, conviértelo en DataFrame
y_train = pd.Series(y_train)  # Si y_train es un ndarray, conviértelo en Series

# Definir el grid de hiperparámetros para GridSearchCV
param_grid = {
    'n_estimators': [200, 300],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 5, 7],
    'min_child_weight': [3, 5],
    'subsample': [0.7, 0.8, 1.0],
    'colsample_bytree': [0.7, 0.9],
    'gamma': [0.1, 0.3],
}

# Crear un modelo de Random Forest
xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42)

# Configuración de GridSearchCV con validación cruzada
grid_search = GridSearchCV(estimator=xgb_model, 
                           param_grid=param_grid, 
                           scoring='neg_mean_absolute_error', 
                           cv=3, n_jobs=-1, verbose=1)

cv = 3  # Número de folds

# Inicializamos listas para guardar las métricas de cada fold
r2_scores = []
mse_scores = []
mae_scores = []
mape_scores = []

# Realizar la validación cruzada
kf = KFold(n_splits=cv, shuffle=True, random_state=42)
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]  # Usar .iloc con DataFrame
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]  # Usar .iloc con Series
    
    # Entrenar el modelo en el fold
    grid_search.fit(X_train_fold, y_train_fold)
    
    # Predecir en el conjunto de validación
    y_val_pred_log = grid_search.predict(X_val_fold)
    
    # Revertir la transformación logarítmica
    y_val_pred = np.expm1(y_val_pred_log)
    y_val_actual = np.expm1(y_val_fold)
    
    # Calcular las métricas para este fold
    r2 = r2_score(y_val_actual, y_val_pred)
    mse = mean_squared_error(y_val_actual, y_val_pred)
    mae = mean_absolute_error(y_val_actual, y_val_pred)
    mape = np.mean(np.abs((y_val_actual - y_val_pred) / y_val_actual)) * 100

    # Guardar las métricas en las listas
    r2_scores.append(r2)
    mse_scores.append(mse)
    mae_scores.append(mae)
    mape_scores.append(mape) 
# Obtener el mejor modelo
best_rf_model = grid_search.best_estimator_

# Obtener los mejores parámetros
best_params = grid_search.best_params_
print("Mejores parámetros:", best_params)

# Promedios de métricas en CV
mean_r2_cv = np.mean(r2_scores)
mean_mse_cv = np.mean(mse_scores)
mean_rmse_cv = np.sqrt(mean_mse_cv)
mean_mae_cv = np.mean(mae_scores)
mean_mape_cv = np.mean(mape_scores) 

# Métricas de Cross-Validation
print("=== Métricas de Cross-Validation ===")
print(f"R² (promedio CV): {mean_r2_cv:.4f}")
print(f"MSE (promedio CV): {mean_mse_cv:.4f}")
print(f"RMSE (promedio CV): {mean_rmse_cv:.4f}")
print(f"MAE (promedio CV): {mean_mae_cv:.4f}")
print(f"MAPE (promedio CV): {mean_mape_cv:.4f}%")

Fitting 3 folds for each of 288 candidates, totalling 864 fits
Fitting 3 folds for each of 288 candidates, totalling 864 fits
Fitting 3 folds for each of 288 candidates, totalling 864 fits


c:\Users\emita\.conda\envs\proyecto_ml\lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Mejores parámetros: {'colsample_bytree': 0.9, 'gamma': 0.1, 'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 3, 'n_estimators': 200, 'subsample': 0.7}
=== Métricas de Cross-Validation ===
R² (promedio CV): 0.5821
MSE (promedio CV): 217752793.6484
RMSE (promedio CV): 14756.4492
MAE (promedio CV): 10461.4388
MAPE (promedio CV): 21.1169%


Pruebo con Gradient Boosting

In [17]:
# Asegúrate de convertir los arrays de numpy a pandas DataFrame y Series si es necesario
X_train = pd.DataFrame(X_train)  # Si X_train es un ndarray, conviértelo en DataFrame
y_train = pd.Series(y_train)  # Si y_train es un ndarray, conviértelo en Series

# --- Definir el Espacio de Búsqueda de Hiperparámetros ---
param_distributions = {
    'n_estimators': [int(x) for x in np.linspace(start=50, stop=500, num=10)],
    'learning_rate': [0.001, 0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5, 6],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'subsample': [0.7, 0.8, 0.9, 1.0],
    'max_features': ['auto', 'sqrt', 'log2'],
    'alpha': [0.1, 0.5, 0.9], #parametro de regularizacion L1
    'ccp_alpha': [0.0, 0.01, 0.1] #parametro para la poda
}
# --- Instanciar el Modelo y RandomizedSearchCV ---
gbr = GradientBoostingRegressor(random_state=42, loss='squared_error') 

random_search = RandomizedSearchCV(
    estimator=gbr,  # Cambiar a GradientBoostingRegressor
    param_distributions=param_distributions,
    n_iter=100,  # Número de combinaciones a probar
    cv=3,  # Número de folds en la validación cruzada
    verbose=2,
    random_state=42,
    n_jobs=-1,  # Usar todos los procesadores disponibles
    scoring='neg_mean_squared_error'  # Puedes usar otra métrica si lo prefieres
)
cv = 5  # Número de folds

# Inicializamos listas para guardar las métricas de cada fold
r2_scores = []
mse_scores = []
mae_scores = []
mape_scores = []

# Realizar la validación cruzada
kf = KFold(n_splits=cv, shuffle=True, random_state=42)
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]  # Usar .iloc con DataFrame
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]  # Usar .iloc con Series
    
    # Entrenar el modelo en el fold
    random_search.fit(X_train_fold, y_train_fold)
    
    # Predecir en el conjunto de validación
    y_val_pred_log = random_search.predict(X_val_fold)
    
    # Revertir la transformación logarítmica
    y_val_pred = np.expm1(y_val_pred_log)
    y_val_actual = np.expm1(y_val_fold)
    
    # Calcular las métricas para este fold
    r2 = r2_score(y_val_actual, y_val_pred)
    mse = mean_squared_error(y_val_actual, y_val_pred)
    mae = mean_absolute_error(y_val_actual, y_val_pred)
    mape = np.mean(np.abs((y_val_actual - y_val_pred) / y_val_actual)) * 100

    # Guardar las métricas en las listas
    r2_scores.append(r2)
    mse_scores.append(mse)
    mae_scores.append(mae)
    mape_scores.append(mape) 
# Obtener el mejor modelo
best_rf_model = random_search.best_estimator_

# Obtener los mejores parámetros
best_params = random_search.best_params_
print("Mejores parámetros:", best_params)

# Promedios de métricas en CV
mean_r2_cv = np.mean(r2_scores)
mean_mse_cv = np.mean(mse_scores)
mean_rmse_cv = np.sqrt(mean_mse_cv)
mean_mae_cv = np.mean(mae_scores)
mean_mape_cv = np.mean(mape_scores) 

# Métricas de Cross-Validation
print("=== Métricas de Cross-Validation ===")
print(f"R² (promedio CV): {mean_r2_cv:.4f}")
print(f"MSE (promedio CV): {mean_mse_cv:.4f}")
print(f"RMSE (promedio CV): {mean_rmse_cv:.4f}")
print(f"MAE (promedio CV): {mean_mae_cv:.4f}")
print(f"MAPE (promedio CV): {mean_mape_cv:.4f}%")

Fitting 3 folds for each of 100 candidates, totalling 300 fits


c:\Users\emita\.conda\envs\proyecto_ml\lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
108 fits failed out of a total of 300.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
40 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\emita\.conda\envs\proyecto_ml\lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\emita\.conda\envs\proyecto_ml\lib\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "c:\Users\emita\.conda\envs\proyecto_ml\lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constr

Fitting 3 folds for each of 100 candidates, totalling 300 fits


c:\Users\emita\.conda\envs\proyecto_ml\lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
108 fits failed out of a total of 300.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
40 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\emita\.conda\envs\proyecto_ml\lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\emita\.conda\envs\proyecto_ml\lib\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "c:\Users\emita\.conda\envs\proyecto_ml\lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constr

Fitting 3 folds for each of 100 candidates, totalling 300 fits


...

Fitting 3 folds for each of 100 candidates, totalling 300 fits


c:\Users\emita\.conda\envs\proyecto_ml\lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
108 fits failed out of a total of 300.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
73 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\emita\.conda\envs\proyecto_ml\lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\emita\.conda\envs\proyecto_ml\lib\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "c:\Users\emita\.conda\envs\proyecto_ml\lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constr

Fitting 3 folds for each of 100 candidates, totalling 300 fits


c:\Users\emita\.conda\envs\proyecto_ml\lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
108 fits failed out of a total of 300.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
44 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\emita\.conda\envs\proyecto_ml\lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\emita\.conda\envs\proyecto_ml\lib\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "c:\Users\emita\.conda\envs\proyecto_ml\lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constr

Mejores parámetros: {'subsample': 0.9, 'n_estimators': 450, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 5, 'learning_rate': 0.1, 'ccp_alpha': 0.0, 'alpha': 0.1}
=== Métricas de Cross-Validation ===
R² (promedio CV): 0.5819
MSE (promedio CV): 218156677.6926
RMSE (promedio CV): 14770.1279
MAE (promedio CV): 10611.1752
MAPE (promedio CV): 21.4742%


Funciona mejor un linear regressor simple que cualquier otro modelo.

Pruebo con SVR

In [22]:
# Asegúrate de convertir los arrays de numpy a pandas DataFrame y Series si es necesario
X_train = pd.DataFrame(X_train)  # Si X_train es un ndarray, conviértelo en DataFrame
y_train = pd.Series(y_train)  # Si y_train es un ndarray, conviértelo en Series

# Definir el espacio de búsqueda de hiperparámetros
param_distributions = {
    'C': np.logspace(-3, 3, 7),
    'kernel': ['rbf', 'poly', 'sigmoid', 'linear'],
    'gamma': [0.001, 0.01, 0.1, 1, 10, 100, 'scale', 'auto'],
    'epsilon': [0.01, 0.1, 0.2, 0.5, 1.0]
}

# Instanciar el modelo SVR
svr = SVR()

# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=svr,
    param_distributions=param_distributions,
    n_iter=100,  # Aumenta este valor si tienes tiempo
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=-1,
    scoring='neg_mean_squared_error'
)

cv = 3  # Número de folds

# Inicializamos listas para guardar las métricas de cada fold
r2_scores = []
mse_scores = []
mae_scores = []
mape_scores = []

# Realizar la validación cruzada
kf = KFold(n_splits=cv, shuffle=True, random_state=42)
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]  # Usar .iloc con DataFrame
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]  # Usar .iloc con Series
    
    # Entrenar el modelo en el fold
    random_search.fit(X_train_fold, y_train_fold)
    
    # Predecir en el conjunto de validación
    y_val_pred_log = random_search.predict(X_val_fold)
    
    # Revertir la transformación logarítmica
    y_val_pred = np.expm1(y_val_pred_log)
    y_val_actual = np.expm1(y_val_fold)
    
    # Calcular las métricas para este fold
    r2 = r2_score(y_val_actual, y_val_pred)
    mse = mean_squared_error(y_val_actual, y_val_pred)
    mae = mean_absolute_error(y_val_actual, y_val_pred)
    mape = np.mean(np.abs((y_val_actual - y_val_pred) / y_val_actual)) * 100

    # Guardar las métricas en las listas
    r2_scores.append(r2)
    mse_scores.append(mse)
    mae_scores.append(mae)
    mape_scores.append(mape) 
# Obtener el mejor modelo
best_rf_model = random_search.best_estimator_

# Obtener los mejores parámetros
best_params = random_search.best_params_
print("Mejores parámetros:", best_params)

# Promedios de métricas en CV
mean_r2_cv = np.mean(r2_scores)
mean_mse_cv = np.mean(mse_scores)
mean_rmse_cv = np.sqrt(mean_mse_cv)
mean_mae_cv = np.mean(mae_scores)
mean_mape_cv = np.mean(mape_scores) 

# Métricas de Cross-Validation
print("=== Métricas de Cross-Validation ===")
print(f"R² (promedio CV): {mean_r2_cv:.4f}")
print(f"MSE (promedio CV): {mean_mse_cv:.4f}")
print(f"RMSE (promedio CV): {mean_rmse_cv:.4f}")
print(f"MAE (promedio CV): {mean_mae_cv:.4f}")
print(f"MAPE (promedio CV): {mean_mape_cv:.4f}%")

Fitting 3 folds for each of 100 candidates, totalling 300 fits


KeyboardInterrupt: 